In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE # 效果也滿差的，只比overSample好一點
from imblearn.combine import SMOTEENN # 效果不錯，比SMOTE好多了
from imblearn.combine import SMOTETomek # 效果挺差的，與SMOTE差不多
from imblearn.under_sampling import RandomUnderSampler # underSample搭配RF效果不錯，再加上pca更好一點

In [2]:
'''
Read input data
'''
train_data = pd.read_csv('training_data.csv', header=None)
test_data = pd.read_csv('testing_data.csv', header=None)
train_size = len(train_data)
test_size = len(test_data)
# print(train_size, test_size) # 47438 25545

In [3]:
# print(train_data)

In [4]:
# print(test_data)

In [5]:
'''
Deal with input data
'''
test_index = test_data.loc[:,0]
test_data = test_data.drop(0, axis=1) #J:看不太懂為何把column[0]drop掉
# print(test_data)

def test_rename(x):
    return x+1
test_data = test_data.rename(mapper=test_rename, axis=1) # 不明原因，這邊把全部的column都+1了
train_label = train_data.iloc[:,1]
train_data_d = train_data.drop(labels=[0,1], axis=1)
# print(train_data_d)


# #J: 這邊亂試試看dropna 怪怪的 好像有bug
# train_data_d = train_data_d.dropna(axis=0)
# print(train_data_d)

In [6]:
concated_df = pd.concat((train_data_d, test_data))
# print(concated_df) #J: concat在一起後，一起處理缺失值、oneHotCoding，接著再重新分開一次

In [7]:
cat_feature = [2,5,6,7,8,9,10,11,13,14,15,24,25,27]
num_feature = [3,4,12,16,17,18,19,20,21,22,23,26,28]

In [8]:
'''
Deal with missing value 可嘗試調整這裡
Fill category feature with mode, numeric feature with mean
'''
# J:原始版本的填值方式
for col in cat_feature:
    concated_df[col] = concated_df[col].fillna(concated_df[col].mode()[0])
for col in num_feature:
    concated_df[col] = concated_df[col].fillna(concated_df[col].mean())
concated_df = pd.get_dummies(concated_df, columns=cat_feature)
    
    
# J:這邊我直接用drop看看。 結果失敗，連test集都有缺漏值
# concated_df = concated_df.dropna(axis=0)
# concated_df = pd.get_dummies(concated_df, columns=cat_feature)

In [9]:
train_data_d = concated_df.iloc[:train_size,:]
test_data = concated_df.iloc[train_size:,:]
# print(len(train_data_d), len(test_data) )

In [10]:
'''
Train/test dataframe into numpy array
'''
X = train_data_d.values
y = train_label.values

'''處理Data Imbalance問題，重要!!!!'''
# 嘗試underSample 效果不錯
# rus = RandomUnderSampler(random_state=42)
# X_resampled, y_resampled = rus.fit_resample(X, y)

# 嘗試SMOTE 效果不太好
# sm = SMOTE(random_state=42)
# X_resampled, y_resampled = sm.fit_sample(X, y)

# 嘗試SMOTEENN 效果比SMOTE好很多 但還是比underSample差
sme = SMOTEENN(random_state=42)
X_resampled, y_resampled = sme.fit_sample(X, y)

# 嘗試SMOTETomek 效果比SMOTEENN差
# smt = SMOTETomek(random_state=42)
# X_resampled, y_resampled = smt.fit_sample(X, y)

X_test = test_data.values # J: 要預測的資料集，不要動到哦!

In [11]:
print(len(X_resampled))

60897


In [12]:
print(len(y_resampled))

60897


In [13]:
print(len(X_test))

25545


In [14]:
'''
Standardlize numeric feature
'''
# 原本的標準化方法
std = StandardScaler()
std.fit(X_resampled[:,:14])
X_resampled[:,:14] = std.transform(X_resampled[:,:14])
X_test[:,:14] = std.transform(X_test[:,:14])


# J: 嘗試標準化更多東西，沒比較厲害
# std = StandardScaler()
# std.fit(X_resampled[:, :])
# X_resampled[:, :] = std.transform(X_resampled[:, :])
# X_test[:, :] = std.transform(X_test[:, :])

In [15]:
# J: 使用PCA降維
pca = PCA(n_components=20)
X_resampled = pca.fit_transform(X_resampled)
X_test = pca.transform(X_test)

In [16]:
'''
Train model
'''
from sklearn.ensemble import RandomForestClassifier # RF效果還不錯 
# 預設使用RF 效果不錯
clf = RandomForestClassifier(n_estimators = 105 , n_jobs = 4,random_state=10)
clf.fit(X_resampled , y_resampled)


# Make prediction
pred = clf.predict(X_test)


# Output prediction
with open('super_easy_baseline_rf.csv','w') as f:
    f.write('Id,Prediction\n')
    for idx , p in zip(test_index , pred):
        f.write('{},{}\n'.format(idx,p))

In [17]:
from sklearn import svm # RBF SVM效果其實不錯
# 嘗試RBF SVM
clf = svm.SVC(kernel='rbf' ,C=2 ,random_state=0) 
clf.fit(X_resampled, y_resampled)


# Make prediction
pred = clf.predict(X_test)


# Output prediction
with open('super_easy_baseline_svm.csv','w') as f:
    f.write('Id,Prediction\n')
    for idx , p in zip(test_index , pred):
        f.write('{},{}\n'.format(idx,p))

C:\Users\shawn\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [18]:
from sklearn.ensemble import AdaBoostClassifier # AdaBoost效果不是挺好
# 嘗試AdaBoost
clf = AdaBoostClassifier(n_estimators=100)
clf.fit(X_resampled , y_resampled)


# Make prediction
pred = clf.predict(X_test)


# Output prediction
with open('super_easy_baseline_ada.csv','w') as f:
    f.write('Id,Prediction\n')
    for idx , p in zip(test_index , pred):
        f.write('{},{}\n'.format(idx,p))

In [19]:
from sklearn.ensemble import BaggingClassifier # Bagging效果也不是挺好
# 嘗試Bagging 
clf = BaggingClassifier(n_estimators = 100)
clf.fit(X_resampled , y_resampled)


# Make prediction
pred = clf.predict(X_test)


# Output prediction
with open('super_easy_baseline_bagging.csv','w') as f:
    f.write('Id,Prediction\n')
    for idx , p in zip(test_index , pred):
        f.write('{},{}\n'.format(idx,p))